In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", 30)

In [2]:
import seaborn as sns

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

In [4]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
sample_submission= pd.read_csv("data/sample_submission.csv")

In [5]:
#데이터 확인

In [6]:
test.shape, train.shape, sample_submission.shape

((400000, 24), (600000, 25), (400000, 2))

In [7]:
from sklearn.experimental import enable_iterative_imputer  # still experimental 
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer 

In [8]:
train_ex=train.drop('target',axis=1)
train_ex

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,Bassoon,de4c57ee2,a64bc7ddf,598080a91,0256c7a4b,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0
1,1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,NaN,Theremin,2bb3c3e5c,3a3a936e8,1dddb8473,52ead350c,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0
2,2,0.0,1.0,0.0,F,N,Red,NaN,Hamster,Canada,Bassoon,b574c9841,708248125,5ddc9a726,745b909d1,NaN,3.0,NaN,Freezing,n,P,eN,5.0,9.0
3,3,NaN,0.0,0.0,F,N,Red,Circle,Hamster,Finland,Theremin,673bdf1f6,23edb8da3,3a33ef960,bdaa56dd1,f9d456e57,1.0,Novice,Lava Hot,a,C,NaN,3.0,3.0
4,4,0.0,NaN,0.0,T,N,Red,Triangle,Hamster,Costa Rica,NaN,777d1ac2c,3a7975e46,bc9cc2a94,NaN,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,599995,0.0,1.0,0.0,T,N,Red,Polygon,Axolotl,India,Theremin,014770cf0,da5014b01,a7059911d,158183c63,015c63324,3.0,Novice,Freezing,a,R,GZ,5.0,NaN
599996,599996,1.0,0.0,0.0,T,Y,Blue,Polygon,Dog,Costa Rica,Oboe,NaN,2023ed4ed,83bdea3a5,e9fde8fa8,a02ae6a63,2.0,Novice,Boiling Hot,n,N,sf,NaN,3.0
599997,599997,0.0,0.0,0.0,F,Y,Red,Circle,Axolotl,Russia,Theremin,c7dc5d460,5d7d341ac,114b1dbf3,cccbca824,40f9610c1,2.0,Contributor,Freezing,n,H,MV,7.0,5.0
599998,599998,1.0,1.0,0.0,F,Y,NaN,Polygon,Axolotl,NaN,Piano,4d7780407,209e1054e,fba315672,4164322bd,c1a8374a0,1.0,Master,Warm,m,X,Ey,1.0,5.0


In [9]:
train.columns

Index(['id', 'bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4', 'nom_0', 'nom_1',
       'nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9',
       'ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5', 'day', 'month',
       'target'],
      dtype='object')

In [10]:
imp = SimpleImputer(strategy="most_frequent")
train_imp = pd.DataFrame(imp.fit_transform(train_ex))
test_imp = pd.DataFrame(imp.transform(test))

In [11]:
train_imp 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,Bassoon,de4c57ee2,a64bc7ddf,598080a91,0256c7a4b,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0
1,1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,India,Theremin,2bb3c3e5c,3a3a936e8,1dddb8473,52ead350c,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0
2,2,0.0,1.0,0.0,F,N,Red,Triangle,Hamster,Canada,Bassoon,b574c9841,708248125,5ddc9a726,745b909d1,8f3276a6e,3.0,Novice,Freezing,n,P,eN,5.0,9.0
3,3,0.0,0.0,0.0,F,N,Red,Circle,Hamster,Finland,Theremin,673bdf1f6,23edb8da3,3a33ef960,bdaa56dd1,f9d456e57,1.0,Novice,Lava Hot,a,C,Fl,3.0,3.0
4,4,0.0,0.0,0.0,T,N,Red,Triangle,Hamster,Costa Rica,Theremin,777d1ac2c,3a7975e46,bc9cc2a94,7d7c02c57,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,599995,0.0,1.0,0.0,T,N,Red,Polygon,Axolotl,India,Theremin,014770cf0,da5014b01,a7059911d,158183c63,015c63324,3.0,Novice,Freezing,a,R,GZ,5.0,8.0
599996,599996,1.0,0.0,0.0,T,Y,Blue,Polygon,Dog,Costa Rica,Oboe,fc8fc7e56,2023ed4ed,83bdea3a5,e9fde8fa8,a02ae6a63,2.0,Novice,Boiling Hot,n,N,sf,3.0,3.0
599997,599997,0.0,0.0,0.0,F,Y,Red,Circle,Axolotl,Russia,Theremin,c7dc5d460,5d7d341ac,114b1dbf3,cccbca824,40f9610c1,2.0,Contributor,Freezing,n,H,MV,7.0,5.0
599998,599998,1.0,1.0,0.0,F,Y,Red,Polygon,Axolotl,India,Piano,4d7780407,209e1054e,fba315672,4164322bd,c1a8374a0,1.0,Master,Warm,m,X,Ey,1.0,5.0


In [82]:
train_imp.rename(columns = {0:'id', 1:'bin_0', 2:'bin_1', 3:'bin_2', 4:'bin_3', 5:'bin_4', 6:'nom_0'
                            , 7:'nom_1',
       8:'nom_2', 9:'nom_3', 10:'nom_4', 11:'nom_5', 12:'nom_6', 13:'nom_7', 14:'nom_8', 15:'nom_9',
       16:'ord_0', 17:'ord_1', 18:'ord_2', 19:'ord_3', 20:'ord_4', 21:'ord_5', 22:'day',23:'month',
       24:'target'}, inplace= True)

In [40]:
#Split into train and test sets

In [82]:
from sklearn.model_selection import train_test_split
X = train.drop('target', axis=1)
y = train['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)